# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 16 2023 8:30AM,255038,10,0086298640,ISDIN Corporation,Released
1,Jan 16 2023 8:30AM,255038,10,0086298641,ISDIN Corporation,Released
2,Jan 16 2023 8:30AM,255038,10,0086298642,ISDIN Corporation,Released
3,Jan 16 2023 8:30AM,255038,10,0086298647,ISDIN Corporation,Released
4,Jan 16 2023 8:30AM,255038,10,0086298654,ISDIN Corporation,Released
5,Jan 16 2023 8:30AM,255044,10,SO94493,"Senseonics, Incorporated",Released
6,Jan 16 2023 8:30AM,255044,10,SO94494,"Senseonics, Incorporated",Released
7,Jan 16 2023 8:30AM,255038,10,0086298635,ISDIN Corporation,Completed
8,Jan 13 2023 5:06PM,255047,16,EMERSA-9028778,"Emersa Waterbox, LLC",Released
9,Jan 13 2023 5:06PM,255046,16,EMERSA-9028777,"Emersa Waterbox, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
7,255038,Released,5
8,255044,Released,2
9,255045,Released,1
10,255046,Released,1
11,255047,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
255038,1.0,NaN,5.0
255044,NaN,NaN,2.0
255045,NaN,NaN,1.0
255046,NaN,NaN,1.0
255047,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255015,0.0,0.0,1.0
255026,0.0,1.0,20.0
255027,0.0,0.0,35.0
255033,0.0,0.0,1.0
255036,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255015,0,0,1
255026,0,1,20
255027,0,0,35
255033,0,0,1
255036,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255015,0,0,1
1,255026,0,1,20
2,255027,0,0,35
3,255033,0,0,1
4,255036,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255015,,,1
1,255026,,1,20
2,255027,,,35
3,255033,,,1
4,255036,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 16 2023 8:30AM,255038,10,ISDIN Corporation
5,Jan 16 2023 8:30AM,255044,10,"Senseonics, Incorporated"
8,Jan 13 2023 5:06PM,255047,16,"Emersa Waterbox, LLC"
9,Jan 13 2023 5:06PM,255046,16,"Emersa Waterbox, LLC"
10,Jan 13 2023 4:16PM,255045,16,Sartorius Stedim Filters Inc.
11,Jan 13 2023 3:47PM,255036,16,Sartorius Stedim Filters Inc.
12,Jan 13 2023 3:12PM,255033,19,"NAPP Technologies, LLC"
13,Jan 13 2023 2:15PM,255027,19,"Innogenix, LLC"
48,Jan 13 2023 1:58PM,255026,20,"Exact-Rx, Inc."
69,Jan 13 2023 12:32PM,255015,18,Innova Softgel LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 16 2023 8:30AM,255038,10,ISDIN Corporation,1,,5
1,Jan 16 2023 8:30AM,255044,10,"Senseonics, Incorporated",,,2
2,Jan 13 2023 5:06PM,255047,16,"Emersa Waterbox, LLC",,,1
3,Jan 13 2023 5:06PM,255046,16,"Emersa Waterbox, LLC",,,1
4,Jan 13 2023 4:16PM,255045,16,Sartorius Stedim Filters Inc.,,,1
5,Jan 13 2023 3:47PM,255036,16,Sartorius Stedim Filters Inc.,,,1
6,Jan 13 2023 3:12PM,255033,19,"NAPP Technologies, LLC",,,1
7,Jan 13 2023 2:15PM,255027,19,"Innogenix, LLC",,,35
8,Jan 13 2023 1:58PM,255026,20,"Exact-Rx, Inc.",,1,20
9,Jan 13 2023 12:32PM,255015,18,Innova Softgel LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 8:30AM,255038,10,ISDIN Corporation,5,,1
1,Jan 16 2023 8:30AM,255044,10,"Senseonics, Incorporated",2,,
2,Jan 13 2023 5:06PM,255047,16,"Emersa Waterbox, LLC",1,,
3,Jan 13 2023 5:06PM,255046,16,"Emersa Waterbox, LLC",1,,
4,Jan 13 2023 4:16PM,255045,16,Sartorius Stedim Filters Inc.,1,,
5,Jan 13 2023 3:47PM,255036,16,Sartorius Stedim Filters Inc.,1,,
6,Jan 13 2023 3:12PM,255033,19,"NAPP Technologies, LLC",1,,
7,Jan 13 2023 2:15PM,255027,19,"Innogenix, LLC",35,,
8,Jan 13 2023 1:58PM,255026,20,"Exact-Rx, Inc.",20,1,
9,Jan 13 2023 12:32PM,255015,18,Innova Softgel LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 8:30AM,255038,10,ISDIN Corporation,5,,1
1,Jan 16 2023 8:30AM,255044,10,"Senseonics, Incorporated",2,,
2,Jan 13 2023 5:06PM,255047,16,"Emersa Waterbox, LLC",1,,
3,Jan 13 2023 5:06PM,255046,16,"Emersa Waterbox, LLC",1,,
4,Jan 13 2023 4:16PM,255045,16,Sartorius Stedim Filters Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 8:30AM,255038,10,ISDIN Corporation,5,NaN,1.0
1,Jan 16 2023 8:30AM,255044,10,"Senseonics, Incorporated",2,NaN,NaN
2,Jan 13 2023 5:06PM,255047,16,"Emersa Waterbox, LLC",1,NaN,NaN
3,Jan 13 2023 5:06PM,255046,16,"Emersa Waterbox, LLC",1,NaN,NaN
4,Jan 13 2023 4:16PM,255045,16,Sartorius Stedim Filters Inc.,1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 8:30AM,255038,10,ISDIN Corporation,5,0.0,1.0
1,Jan 16 2023 8:30AM,255044,10,"Senseonics, Incorporated",2,0.0,0.0
2,Jan 13 2023 5:06PM,255047,16,"Emersa Waterbox, LLC",1,0.0,0.0
3,Jan 13 2023 5:06PM,255046,16,"Emersa Waterbox, LLC",1,0.0,0.0
4,Jan 13 2023 4:16PM,255045,16,Sartorius Stedim Filters Inc.,1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,510082,7,0.0,1.0
16,1020174,4,0.0,0.0
18,255015,1,0.0,0.0
19,510060,36,0.0,0.0
20,255026,20,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,510082,7,0.0,1.0
1,16,1020174,4,0.0,0.0
2,18,255015,1,0.0,0.0
3,19,510060,36,0.0,0.0
4,20,255026,20,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,7,0.0,1.0
1,16,4,0.0,0.0
2,18,1,0.0,0.0
3,19,36,0.0,0.0
4,20,20,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,7.0
1,16,Released,4.0
2,18,Released,1.0
3,19,Released,36.0
4,20,Released,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20
Status,,,,,
Completed,1.0,0.0,0.0,0.0,0.0
Executing,0.0,0.0,0.0,0.0,1.0
Released,7.0,4.0,1.0,36.0,20.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,0.0,1.0
2,Released,7.0,4.0,1.0,36.0,20.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,0.0,0.0,1.0
2,Released,7.0,4.0,1.0,36.0,20.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()